## Drought and Crop Yield <a class="anchor" id="introduction"></a>


#### Project Proposal:
Throughout the years, droughts have received more attention especially by weather specialists. With climate change being actively tracked, it is important for us to understand drought patterns and how they will relate to crop yield, which will help us understand global food security for the coming years. Recently, there has been a growing food demand with an increase in the world population along with drastic changes in the weather patterns. Through this study, we plan to help farmers understand the extreme changes within weather patterns and how it will impact crop yield by understanding previous crop yield and loss patterns. 


#### Proposed Project: 
We will understand the relationship trend between crop yield and drought pattern in North and South America, and try to comprehend which crops are less correlated to drought data indications. Throughout this project, we will want to get a better knowledge of drought impact on agriculture and to extend further, how crops are currently affected by climate change. 

#### Questions to answer:
* Which crops are less correlated to drought data indications? If we have some crops that are less correlated, will this crop be a ‘good-yield’ crop in the areas that are more prone to drought? 
* How has climate change impacted crop yields over the last few years and analyzing whether there were any extreme changes within the US crop yield patterns?
* As an addition, we would also want to understand the soil moisture data in correspondence to drought patterns and how soil moisture affects crop yields.

#### Scope of Study:
* Location: We will study this crop yield and drought indication relation across US states
* Timeframe: according to limitation of data, We will need to focus on study of data from year 2010 to 2020

#### Limitations: 
Analyzing crop yield data requires tons and tons of data such as soil moisture, temperature, use of fertilizers. For better analysis and performance, it is important to obtain as much data as possible. 


In [ ]:
import os
import re
import csv
import glob
import matplotlib
import numpy as np
import geopandas as gpd
import pandas as pd
import folium
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
import altair as alt
from datetime import datetime
from functools import reduce
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.manifold import MDS
from sklearn.preprocessing import LabelEncoder

warnings.simplefilter("ignore")
%matplotlib inline
# import ee

# set max df column display
pd.set_option('display.max_columns', 500)

### Primary Dataset <a class="anchor" id="primary"></a>

our crop dataset from https://www.nass.usda.gov/Statistics_by_Subject/index.php?sector=CROPS

In [ ]:
# get all the data together
def getframe(folderpath, axis=0):
    path = folderpath
    all_files = glob.glob(os.path.join(path, "*.csv"))
    df_from_each_file = (pd.read_csv(f) for f in all_files)
    concat_df   = pd.concat(df_from_each_file, axis=axis, ignore_index=True)
    return concat_df

# crop_df = getframe('drive/MyDrive/Colab Notebooks/data/crop_data/')
crop_df = getframe('crops_datasets_raw')

In [ ]:
crop_df.head(3)

### Secondary Dataset <a class="anchor" id="secondary_dataset"></a>
#### Now let's get our drought data from USA Drought Monitor website

Source: https://droughtmonitor.unl.edu/DmData/DataDownload/ComprehensiveStatistics.aspx 

The file will be loaded in a csv format from the above website using the following parameters:

* Start Date: 01/01/2010 and End Date: 12/31/2020
* Spatial Scale: State and choose all states
* Statistics Category: Reports at percent level one drought category at time

To understand the data better, we could use the data dictionary present in the website itself: 
*   None is no reported drought
*   D0 - Abnormally dry
*   D1 - Moderate Drought
*   D2 - Severe Drought
*   D3 - Extreme Drought
*   D4 - Exceptional Drought.

These drought variables are calculated using various data such as precipitation, soil moisuture, surface temperature being the main variables.







In [ ]:
# drought_df_raw = pd.read_csv('drive/MyDrive/Colab Notebooks/data/drought_data/drought_data_2010-2020.csv')
drought_df_raw = pd.read_csv('secondary_dataset/drought_data.csv')

In [ ]:
# get date to Datetime format
# find dict for state abbreviation to lookup key and value then merge with the crop file

us_state_to_abbrev = {
    "Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR", "California": "CA", "Colorado": "CO",
    "Connecticut": "CT", "Delaware": "DE", "Florida": "FL", "Georgia": "GA", "Hawaii": "HI", "Idaho": "ID", "Illinois": "IL", "Indiana": "IN",
    "Iowa": "IA", "Kansas": "KS", "Kentucky": "KY", "Louisiana": "LA", "Maine": "ME", "Maryland": "MD", "Massachusetts": "MA", "Michigan": "MI",
    "Minnesota": "MN", "Mississippi": "MS", "Missouri": "MO", "Montana": "MT", "Nebraska": "NE", "Nevada": "NV", "New Hampshire": "NH",
    "New Jersey": "NJ", "New Mexico": "NM", "New York": "NY", "North Carolina": "NC", "North Dakota": "ND", "Ohio": "OH", "Oklahoma": "OK",
    "Oregon": "OR", "Pennsylvania": "PA", "Rhode Island": "RI", "South Carolina": "SC", "South Dakota": "SD", "Tennessee": "TN",
    "Texas": "TX", "Utah": "UT", "Vermont": "VT", "Virginia": "VA", "Washington": "WA", "West Virginia": "WV", "Wisconsin": "WI", "Wyoming": "WY",
    "District of Columbia": "DC", "American Samoa": "AS", "Guam": "GU", "Northern Mariana Islands": "MP", "Puerto Rico": "PR",
     "United States Minor Outlying Islands": "UM", "U.S. Virgin Islands": "VI",
}
    
# invert the dictionary
abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))

drought_df_raw['state'] = drought_df_raw['StateAbbreviation'].map(abbrev_to_us_state)
drought_df_raw['MapDate'] = pd.to_datetime(drought_df_raw['MapDate'], format='%Y%M%d')
drought_df_raw.set_index('MapDate', inplace=True)

In [ ]:
drought_year_df = drought_df_raw.groupby([pd.Grouper(freq='Y'), 'state']).mean()
drought_year_df = drought_year_df.reset_index()
drought_year_df['Year'] = drought_year_df['MapDate'].dt.year
drought_year_df.rename(columns={'state':'State'}, inplace=True)
drought_year_df.shape

#### Precipitation by State from 2010-2022
Source : https://www.ncdc.noaa.gov/cag/statewide/time-series

* First get the data from the source above and concat them together.
* The data that we get for this project is monthly data by state from year 2010 to year 2022

In [ ]:
# read each file and change column value to each state
import ntpath

# path = 'drive/MyDrive/Colab Notebooks/data/precipitation/'
path = 'secondary_dataset/precipitation'
all_files = glob.glob(os.path.join(path, "*.csv"))
files_list = []
for file in all_files:
    df = pd.read_csv(file, skiprows=4)
    y = file.rsplit('.', 1)
    x = y[0].rsplit('pcp_', 1)
    df['state'] = x[-1]
    files_list.append(df)
pcp_df = pd.concat(files_list, axis=0, ignore_index=True)
pcp_df['Date'] = pd.to_datetime(pcp_df.Date, format='%Y%m')
pcp_df.set_index('Date', inplace=True)
pcp_df.head(5)

In [ ]:
# pcp_df by year average value
pcp_year_df = pcp_df.groupby([pd.Grouper(freq='Y'), 'state']).mean()
pcp_year_df = pcp_year_df.reset_index()
pcp_year_df['Year'] =pcp_year_df['Date'].dt.year
pcp_year_df.rename(columns={'Value':'precip', 'state':'State'}, inplace=True)
pcp_year_df

#### Land Temperature by State
Source : https://www.ncdc.noaa.gov/cag/statewide/time-series

* First get the data from the source above and concat them together.
* The data that we get for this project is monthly data by state from year 2010 to year 2022

In [ ]:
# read and write csv to collect state from the top of the cell
# path = 'drive/MyDrive/Colab Notebooks/data/temperature/'
path = 'secondary_dataset/temperature'
files = [os.path.join(path, f) for f in os.listdir(path)] # if os.path.isfile(os.path.join(path, f))
df_list = []
for f in files:
    file = open(f, 'r')
    firstline = file.readline()
    state = firstline.split(',')[0] # extract state from csv file
    df = pd.read_csv(f, skiprows=4) # read_csv to df 
    df['state'] = state
    df_list.append(df)
temp_df = pd.concat(df_list, ignore_index=True)
temp_df['Date'] = pd.to_datetime(temp_df['Date'], format='%Y%M')
temp_df.set_index('Date', inplace=True)
temp_df

In [ ]:
temp_year_df = temp_df.groupby([pd.Grouper(freq='Y'), 'state']).mean()
temp_year_df = temp_year_df.reset_index()
temp_year_df['Year'] =temp_year_df['Date'].dt.year
temp_year_df.rename(columns={'Value':'temp', 'state':'State'}, inplace=True)

#### Palmer Drought Severity Index (PDSI)
Source : https://www.ncdc.noaa.gov/cag/statewide/time-series

In [ ]:
# read and write csv to collect state from the top of the cell
# path = 'drive/MyDrive/Colab Notebooks/data/pdsi'
path = 'secondary_dataset/pdsi'
files = [os.path.join(path, f) for f in os.listdir(path)] # if os.path.isfile(os.path.join(path, f))
df_list = []
for f in files:
    file = open(f, 'r')
    firstline = file.readline()
    state = firstline.split(',')[0] # extract state from csv file
    df = pd.read_csv(f, skiprows=3) # read_csv to df 
    df['state'] = state
    df_list.append(df)
pdsi_df = pd.concat(df_list, axis=0, ignore_index=True)
pdsi_df['Date'] = pd.to_datetime(pdsi_df['Date'], format='%Y%M')
pdsi_df.set_index('Date', inplace=True)
pdsi_df.head(5)


In [ ]:
pdsi_year_df = pdsi_df.groupby([pd.Grouper(freq='Y'), 'state']).mean()
pdsi_year_df = pdsi_year_df.reset_index()
pdsi_year_df['Year'] =pdsi_year_df['Date'].dt.year
pdsi_year_df.rename(columns={'Value':'pdsi', 'state':'State'}, inplace=True)
pdsi_year_df

## **Data Cleaning and Manipulation** 

**Step 1**

* For our primary dataset from the crops, we first want to look at the top 10 commodoties in the U.S. We will need to visualize this data at a commodity level rather than at state-level. We will then get the top 10 commodities by filtering out by data-item on acres harvested or produced.

**Step 2**

Create a bar graph for top 10 Commodities in altair with an interaction by year

In [ ]:
def top_commodities():
    filter1 = crop_df.copy()

    filter1 = filter1.drop(columns = ['Program', 'Period','Week Ending','Geo Level','State ANSI','Ag District',
                                    'Ag District Code','County','County ANSI','Zip Code','Region', 'watershed_code',
                                    'Watershed','Domain','Domain Category', 'CV (%)'])

    filter1 = filter1[filter1["Commodity"].str.contains("CROP TOTALS") == False]
    filter1 = filter1[filter1["Commodity"].str.contains("RENT") == False]
    commodity = filter1['Commodity'].unique() # Check to Remove commodities that are totals

    filter1 = filter1[filter1["Data Item"].str.contains("ACRES HARVESTED") == True]
    filter1 = filter1[filter1["Data Item"].str.contains("EXCL ALFALFA") == False]
    data_items = filter1['Data Item'].unique() # Check to retain only acres harvested

    # Convert Values to integer to add and remove any rows that
    filter1['Value'] = filter1['Value'].str.replace(r'[()]',"to_remove")

    # Remove any characters that are not numbers from the value column as it isn't needed
    filter1_f = filter1[filter1["Value"].str.contains("to_remove") == True] ## Do not need
    filter1_t = filter1[filter1["Value"].str.contains("to_remove") == False]

    # Convert Value column into an integer
    filter1_t['Value'] = filter1_t['Value'].str.replace(',', '')
    filter1_t['Value'] = filter1_t['Value'].astype(str).astype(int)
    filter1_t = filter1_t.reset_index(drop=True)

    # Groupby commodity and sum the acres harvested in thousands
    filter2 = filter1_t.groupby(['Commodity'])['Value'].mean().sort_values(ascending=False).reset_index()
    filter2 = filter2.rename(columns = {'Value':'Average Acres Harvested (0000s)'})
    filter2['Average Acres Harvested (0000s)'] = filter2['Average Acres Harvested (0000s)'].div(10000).round(2)

    return filter2


In [ ]:
alt.themes.enable('fivethirtyeight')

source = top_commodities()


bar = alt.Chart(source, title='Top 10 Commodities within 2010 - 2020').mark_bar().encode(
    x='Average Acres Harvested (0000s):Q',
    y=alt.Y('Commodity:N', sort='-x')
).transform_window(
    rank='rank(Average Acres Harvested (0000s))',
    sort=[alt.SortField('Average Acres Harvested (0000s)', order='descending')]
).transform_filter(
    (alt.datum.rank < 11)
)

bar


### Merging the Datasets

We also wants to merge the crop data with all other variables that we have as well as making it a GeoDataframe for easy choropleth plots.

In [ ]:
# state shape file df
# path = 'drive/MyDrive/Colab Notebooks/data/states_shapefile/cb_2018_us_state_500k.shx'
path = 'states_shapefile/cb_2018_us_state_500k.shx'
geo_gdf = gpd.read_file(path)
geo_gdf['NAME'] = geo_gdf['NAME'].str.upper()
geo_gdf.rename(columns={'NAME':'State'}, inplace=True)
geo_gdf.State = geo_gdf.State.str.capitalize()
geo_gdf = geo_gdf[['State', 'geometry']]
geo_gdf.shape

#### Cleaning the Primary Crop Data into Acre, Value ($), and Weight Harvested

In [ ]:
# filter the crop_df1 to only acres_harvested items and only interested columns
crop_df1 = crop_df.copy()
crop_df1 = crop_df1[(crop_df1['Data Item'].str.contains('ACRES HARVESTED') == True)&(crop_df1['Year'] >= 2010)&(crop_df1['Year'] <= 2020)\
                    &(~crop_df1['Commodity'].str.contains('FIELD CROP TOTALS'))]
crop_df1 = crop_df1[['Year', 'State', 'Commodity', 'Value']]
crop_df1.Value = crop_df1.Value.apply(lambda x : x.replace(',', ''))
crop_df1.Value = pd.to_numeric(crop_df1.Value, errors='coerce').fillna(0)

In [ ]:
# group by crop and year, sum value
crop_df1 = crop_df1.groupby(['Year', 'State', 'Commodity']).sum()
crop_df1.reset_index(inplace=True)
crop_df1.State = crop_df1.State.str.capitalize()
crop_df1.rename(columns={'Value':'Acre'}, inplace=True)
crop_df1

In [ ]:
# filter crop data item by production measured in $
crop_df2 = crop_df.copy()
crop_df2 = crop_df2[(crop_df2['Data Item'].str.contains('PRODUCTION, MEASURED IN \$') == True) &(crop_df2['Year'] >= 2010)&(crop_df2['Year'] <= 2020)\
                   &(~crop_df2['Commodity'].str.contains('CROP TOTALS'))] 
crop_df2 = crop_df2[['Year', 'State', 'Commodity', 'Value']]
crop_df2.Value = crop_df2.Value.apply(lambda x : x.replace(',', ''))
crop_df2.Value = pd.to_numeric(crop_df2.Value, errors='coerce').fillna(0)
crop_df2.rename(columns={'Value':'Value_$'}, inplace=True)
crop_df2 = crop_df2.groupby(['Year', 'State', 'Commodity']).sum()
crop_df2.reset_index(inplace=True)
crop_df2.State = crop_df2.State.str.capitalize()
crop_df2.rename(columns={'Value':'Acre'}, inplace=True)
crop_df2

In [ ]:
crop_df3 = crop_df.copy()

# looking at what filter we would need to use and what different unit do we have that we need to transform into weight single unit
crop_df3 = crop_df3[crop_df3['Data Item'].str.contains('PRODUCTION, MEASURED IN')]
units = crop_df3['Data Item'].unique().tolist()
new_unit = set([unit.split('-')[1].strip() for unit in units])
new_unit

unit_list = ['PRODUCTION, MEASURED IN 480 LB BALES',
'PRODUCTION, MEASURED IN BU',
'PRODUCTION, MEASURED IN CWT',
'PRODUCTION, MEASURED IN GALLONS',
'PRODUCTION, MEASURED IN LB',
'PRODUCTION, MEASURED IN TONS']

# first filter the unit above in the data items then convert the unit to tons
# crop_df3 = crop_df3[crop_df3['Data Item'].str.contains(('|').join(unit_list))]
crop_df3 = crop_df3[crop_df3['Data Item'].str.contains('PRODUCTION, MEASURED IN TONS')]
crop_df3 = crop_df3[['Year', 'State', 'Commodity', 'Data Item', 'Value']]
crop_df3.Value = crop_df3.Value.apply(lambda x : x.replace(',', ''))
crop_df3.Value = pd.to_numeric(crop_df3.Value, errors='coerce').dropna()

# crop_df3['Value'] = np.where(crop_df3['Data Item'].str.contains('PRODUCTION, MEASURED IN 480 LB BALES'), crop_df3['Value']*480/2000, crop_df3['Value']) #convert the unit into pounds then tons
# crop_df3['Value'] = np.where(crop_df3['Data Item'].str.contains('PRODUCTION, MEASURED IN BU'), crop_df3['Value']*0.021772, crop_df3['Value']) 
# crop_df3['Value'] = np.where(crop_df3['Data Item'].str.contains('PRODUCTION, MEASURED IN CWT'), crop_df3['Value']*0.056, crop_df3['Value'])
# crop_df3['Value'] = np.where(crop_df3['Data Item'].str.contains('PRODUCTION, MEASURED IN GALLONS'), crop_df3['Value']*11.358/2000, crop_df3['Value']) # convert syrub gallon to pounds then tons
crop_df3 = crop_df3[['Year', 'State', 'Commodity', 'Value']]
crop_df3.rename(columns={'Value':'Tons'}, inplace=True)
crop_df3.State = crop_df3.State.str.capitalize()
crop_df3 = crop_df3.groupby(['Year', 'State', 'Commodity']).mean().reset_index()
crop_df3

In [ ]:
# merge 2 crops value by Acre and by Value
all_crop_df = pd.merge(crop_df1, crop_df2, on=['Year', 'State', 'Commodity'], how='outer')
all_crop_df_value = pd.merge(all_crop_df, crop_df3, on=['Year', 'State', 'Commodity'], how='outer')
all_crop_df_value = all_crop_df_value.dropna()
all_crop_df_value

#### Merging crop yield variables with drought variables

In [ ]:
# merge crop and all variables on year and state
dfs = [all_crop_df_value, drought_year_df, pdsi_year_df, pcp_year_df, temp_year_df]
df = reduce(lambda left,right : pd.merge(left, right, on=['Year', 'State'], how='left'), dfs).fillna(0)
df = df[['Year', 'State', 'Commodity', 'Value_$','Acre', 'Tons','None', 'D0', 'D1', 'D2', 'D3', 'D4', 'pdsi', 'precip', 'temp']]
df['Year'] = pd.to_datetime(df.Year, format='%Y')
df['Commodity'] = df['Commodity'].replace('HAY & HAYLAGE', 'HAY')
df

#### Group states into regions

In [ ]:
# create a geodataframe to work with geopandas or folium map
new_gdf = pd.merge(df, geo_gdf, on='State', how='left')
gdf = gpd.GeoDataFrame(new_gdf)
gdf

df_region = df.copy()
df_region['State'].unique()

df_region['Region']  = df_region['State'].replace(['Alabama','Florida','Georgia','South carolina'],'Southeast', inplace=True)
df_region['Region']  = df_region['State'].replace(['Montana', 'Idaho', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 'Arizona','New mexico'],'Mountain', inplace=True)
df_region['Region']  = df_region['State'].replace(['Arkansas','Louisiana','Mississippi'],'Delta', inplace=True)
df_region['Region']  = df_region['State'].replace(['California', 'Oregon','Washington'],'Pacific Coast', inplace=True)
df_region['Region']  = df_region['State'].replace(['Maine','Vermont','New york','Massachusetts', 'Connecticut' , 'Rhode island', 'New hampshire', 'New jersey','Pennsylvania', 'Delaware', 'Maryland'],'Northeast', inplace=True)
df_region['Region']  = df_region['State'].replace(['North dakota', 'South dakota', 'Nebraska', 'Kansas'],'Northern Plains', inplace=True)
df_region['Region']  = df_region['State'].replace(['Kentucky','North carolina','Tennessee','Virginia','West virginia'],'Appalachian', inplace=True)
df_region['Region']  = df_region['State'].replace(['Michigan', 'Minnesota', 'Wisconsin'],'Lake States', inplace=True)
df_region['Region']  = df_region['State'].replace(['Texas','Oklahoma'],'Southern Plains', inplace=True)
df_region['Region']  = df_region['State'].replace(['Illinois', 'Indiana', 'Iowa', 'Ohio','Missouri'], 'Corn Belt', inplace=True)

df_region = df_region.drop(columns=['Region'])



gdf_region = new_gdf.copy()
gdf_region['Region']  = gdf_region['State'].replace(['Alabama','Florida','Georgia','South carolina'],'Southeast', inplace=True)
gdf_region['Region']  = gdf_region['State'].replace(['Montana', 'Idaho', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 'Arizona','New mexico'],'Mountain', inplace=True)
gdf_region['Region']  = gdf_region['State'].replace(['Arkansas','Louisiana','Mississippi'],'Delta', inplace=True)
gdf_region['Region']  = gdf_region['State'].replace(['California', 'Oregon','Washington'],'Pacific Coast', inplace=True)
gdf_region['Region']  = gdf_region['State'].replace(['Maine','Vermont','New york','Massachusetts', 'Connecticut' , 'Rhode island', 'New hampshire', 'New jersey','Pennsylvania', 'Delaware', 'Maryland'],'Northeast', inplace=True)
gdf_region['Region']  = gdf_region['State'].replace(['North dakota', 'South dakota', 'Nebraska', 'Kansas'],'Northern Plains', inplace=True)
gdf_region['Region']  = gdf_region['State'].replace(['Kentucky','North carolina','Tennessee','Virginia','West virginia'],'Appalachian', inplace=True)
gdf_region['Region']  = gdf_region['State'].replace(['Michigan', 'Minnesota', 'Wisconsin'],'Lake States', inplace=True)
gdf_region['Region']  = gdf_region['State'].replace(['Texas','Oklahoma'],'Southern Plains', inplace=True)
gdf_region['Region']  = gdf_region['State'].replace(['Illinois', 'Indiana', 'Iowa', 'Ohio','Missouri'], 'Corn Belt', inplace=True)

gdf = gpd.GeoDataFrame(gdf_region)
gdf.head(5)


df_region_groupby = df_region.groupby(['Year','State'])['None','D0','D1','D2','D3','D4','pdsi','precip','temp'].mean().reset_index()
df_region_groupby


In [ ]:
gdf

## Crop and Drought Data Analysis
<!-- What do we want to know from the analysis and how can we tell the story from those finding -->
The questions we want answer are, how do crop response to different drought variables? What crop are more tolerance to drought relative to others? How would you re-clustering US state based on drought and climate data?

1. let's first explore each drought variable in chart
2. then let's explore top 5 crops in US small multiples choropleth over 5 years periods
3. crops correlation with each variables
4. US state clustering by drought and climate variables, we can work on monthly data of each varibles

In [ ]:
# group by year and crops
df_crops = df.copy()
df_crops = df_crops.groupby(['Year', 'Commodity'])['Tons'].sum().reset_index()


In [ ]:
# Let's take a peak at varibles vis but let's also plob in crops by states
alt.data_transformers.disable_max_rows() # disable max row error

# selection
selection = alt.selection_multi(fields=['State'], bind='legend')

#base altair with df
base = alt.Chart(df_region_groupby)

crop_norm = alt.Chart(df_crops).mark_area().encode(
    x="Year:T",
    y=alt.Y("Tons:Q", stack="normalize"),
    color=alt.Color("Commodity:N", legend=alt.Legend(orient='right'))
)

crop_bar = alt.Chart(df_crops).mark_bar().encode(
    x="Year:T",
    y=alt.Y("Tons:Q"),
    color=alt.Color("Commodity:N", legend=alt.Legend(orient='right'))
)

pdsi = base.mark_line().encode(
    x=alt.X('Year:T'),
    y=alt.Y('pdsi:Q'),
    color=alt.Color('State:N', legend=alt.Legend(orient='right')),
    tooltip=['State','D4']
).add_selection(selection)

In [ ]:
crop_bar

In [ ]:
crop_norm

In [ ]:
pdsi

In [ ]:
# Let's see if we can average variables across states
# all the variable group only by year and average the results
avg_df = df.groupby('Year').mean().reset_index()
corr_df = avg_df.corr()
corr_df

In [ ]:
def corrmatrix(df, method='pearson'):
    # corr_df_stack = df.corr(method).stack().reset_index().rename(columns={0: 'correlation', 'level_0': 'variable', 'level_1': 'variable2'})
    corr_df_stack = df.corr(method).reset_index().melt('index').rename(columns={'index': 'variable', 'variable': 'variable2', 'value': 'correlation'})
    corr_df_stack['correlation_label'] = corr_df_stack['correlation'].map('{:.2f}'.format).fillna(0)
    sort=df.columns.tolist() 
    # create altair matrix
    base = alt.Chart(corr_df_stack).encode(
        x=alt.X('variable:O', sort=sort, title=None, axis=alt.Axis(labelFontSize=16)),
        y=alt.Y('variable2:O', sort=sort, title=None, axis=alt.Axis(labelFontSize=16))
    )

    # text
    text = base.mark_text(size=14).encode(
        text='correlation_label:N',
        color=alt.condition(alt.datum.correlation > 0.5, alt.value('white'), alt.value('black'))
    )

    # heatmap
    hm = base.mark_rect().encode(
        color='correlation:Q'
    )

    corr_chart = (hm + text).properties(
        width=600,
        height=600
    )
    return corr_chart

In [ ]:
corrmatrix(corr_df)

In [ ]:
# let's look at the pairplot with seaborn
df_pairplot = df[['Value_$', 'Acre', 'Tons', 'None', 'D0', 'D1', 'D2', 'D3', 'D4', 'pdsi', 'precip', 'temp']]
# sns.pairplot(df_pairplot, kind ='kde')

In [ ]:
sns.pairplot(data = df_pairplot,
             y_vars=['None', 'D0', 'D1', 'D2', 'D3', 'D4', 'pdsi', 'precip', 'temp'],
             x_vars=['Value_$','Acre','Tons'])

### What crops are less affected by drought?
As we can now see the relationship of all the variables, now we can see which crop variable would be a good representation of the yield. Here we will use crop production yield in weight to compare among different crops.

In [ ]:
df['Commodity'] = df['Commodity'].replace('HAY & HAYLAGE', 'HAY')
df

In [ ]:
# compare crop yield correlation
crop_avg_df = df.groupby(['Year', 'Commodity'])['Acre','Tons','None','D0',	'D1','D2','D3','D4','pdsi','precip','temp'].mean().reset_index()
crop_avg_df = crop_avg_df.pivot(index='Year', columns='Commodity', values='Tons').reset_index()
crop_drought_corr = crop_avg_df.merge(avg_df[['Year', 'D3', 'pdsi', 'precip', 'temp']], on='Year')
crop_drought_corr = crop_drought_corr.corr()
crop_drought_corr

In [ ]:
# display vis
corrmatrix(crop_drought_corr)

In [ ]:
crop_drought_corr.sort_values('pdsi').index

From this correlation matrix of each crop yields in tons to drought indicator, PDSI, we can see that the rank from least correlated are as follow.
1. Sugarbeets
2. Cotton
3. Hay
4. Corn
5. Sugarcane
6. Sorghum

## Explore Clustering of States Based on Variables
using K-Means to explore how each states would be clustered if we were to based on PDSI, Temperature and Precipitation variables. We first need to explore what is the optimal number of clustering using elbow method

In [ ]:
# Let's explore the clustering of drought and climate in state regions using K-Means
# first let's find optimal number of cluster using elbow methods

# stardize the data
df_clustering = df.copy()
df_clustering = df[['State','pdsi', 'temp', 'precip']]
X = StandardScaler().fit_transform(df_clustering.iloc[:, 1:])

sse=[] # sum of square error
list_k = list(range(1, 10))

for k in list_k:
    km = KMeans(n_clusters=k, init='k-means++', max_iter=300, random_state=0)
    km.fit(X)
    sse.append(km.inertia_)

plt.figure(figsize=(6, 6))
plt.plot(list_k, sse, '-o')
plt.xlabel(r'Number of clusters *k*')
plt.ylabel('Sum of squared distance');

### No definite optimal K cluster found using elbow method

We need to find optimal cluster using silhouette score instead

In [ ]:
# scaled features
df_clustering = df.copy()
df_clustering = df[['State','pdsi', 'temp', 'precip']]

# removing outlier from df_clustering keep only the ones that are within +2.5 to -2.5 standard deviations in the column 'Data'.
df_clustering[np.abs(df_clustering.pdsi-df_clustering.pdsi.mean()) <= (2.5*df_clustering.pdsi.std())]
df_clustering[np.abs(df_clustering.temp-df_clustering.temp.mean()) <= (2.5*df_clustering.temp.std())]
df_clustering[np.abs(df_clustering.precip-df_clustering.precip.mean()) <= (2.5*df_clustering.precip.std())]
df_clustering

# scale the data
X = StandardScaler().fit_transform(df_clustering.iloc[:,1:])
df_scaled = pd.DataFrame(X, columns=['pdsi', 'temp', 'precip'])

# label encoding
states = df_clustering.State.tolist()
state_label = LabelEncoder().fit_transform(states)

# merge label with scaled features
df_scaled['state_label'] = state_label

In [ ]:
for i, k in enumerate([2, 3, 4, 5, 6]):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)
    
    # Run the Kmeans algorithm
    kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=300, random_state=0)
    labels = kmeans.fit_predict(X)
    # centroids = kmeans.cluster_centers_

    # Get silhouette samples
    silhouette_vals = silhouette_samples(X, labels)

    # Silhouette plot
    y_ticks = []
    y_lower, y_upper = 0, 0
    for i, cluster in enumerate(np.unique(labels)):
        cluster_silhouette_vals = silhouette_vals[labels == cluster]
        cluster_silhouette_vals.sort()
        y_upper += len(cluster_silhouette_vals)
        ax1.barh(range(y_lower, y_upper), cluster_silhouette_vals, edgecolor='none', height=1)
        ax1.text(-0.03, (y_lower + y_upper) / 2, str(i + 1))
        y_lower += len(cluster_silhouette_vals)

    # Get the average silhouette score and plot it
    avg_score = np.mean(silhouette_vals)
    ax1.axvline(avg_score, linestyle='--', linewidth=2, color='green')
    ax1.set_yticks([])
    ax1.set_xlim([-0.1, 1])
    ax1.set_xlabel('Silhouette coefficient values')
    ax1.set_ylabel('Cluster labels')
    ax1.set_title('Silhouette plot for the various clusters', y=1.02);
    
    # Use MDS to flatten the data
    embedding = MDS(n_components=2)
    mds = pd.DataFrame(embedding.fit_transform(X), columns = ['component1','component2'])
    mds['labels'] = kmeans.predict(X)

    # Scatter plot of data colored with labels
    ax2.scatter(mds['component1'], mds['component2'], c=labels)
    # ax2.scatter(centroids[:, 0], centroids[:, 1], marker='*', c='r', s=250)
    # ax2.set_xlim([-2, 2])
    # ax2.set_xlim([-2, 2])
    ax2.set_xlabel('Component 1')
    ax2.set_ylabel('Component 2')
    ax2.set_title('Visualization of clustered data', y=1.02)
    ax2.set_aspect('equal')
    plt.tight_layout()
    plt.suptitle(f'Silhouette analysis using k = {k}',
                 fontsize=16, fontweight='semibold', y=1.05);

### Optimal K-Cluster for K-Means using Silhuette Score
with the silhuette score chart above, with average mean being higher than other and not too few a cluster, k-clustering = 5 or 6 is good for our analysis

In [ ]:


# using kmean clustering to find state cluster based on variables
kmeans = KMeans(n_clusters=5, init='k-means++', max_iter=300, random_state=0).fit(X)

# sns.scatterplot(data=mds, x = "component1", y="component2", hue="cluster")

df_scaled['clustering'] = kmeans.predict(X)
df_scaled['State'] = df_clustering['State']
df_scaled


### Plot the clustering result with folium and geopandas
uncomment the code below to plot the choropleth of the clustering by states

In [ ]:
# state_group_gdf = pd.merge(df_scaled, geo_gdf, on='State', how='left')
# gdf = gpd.GeoDataFrame(state_group_gdf)
# m = folium.Map([43, -100], zoom_start=4, tiles='Stamen Terrain')
# geo = gdf.explore(column='clustering', m=m, cmap='gnuplot', legend=True, scheme='EqualInterval', k=5)
# geo